# Assignment 5

## 1.复习课上内容， 阅读相应论文。

## 2. 回答以下理论题目

### 2. 1.  What is autoencoder?
自编码器是一种把输入的数据重构的神经网络。比如输入的是一张人脸的图片，那么在encoder部分，神经网络首先会把人脸的图片变成一个vector，然后decoder部分是把这个vector再变成一个人脸。Autoencoder也可以是一种特征提取的工具，如果用线性方程，那么作用和PCA很接近；如果用非线性的方程，特征提取的内容可能可以更加丰富。用Autoencoder去提取特征是无监督的降维。

Autoencoder 有多种类型，分为：

- Undercomplete autoencoders

- Sparse autoencoders

- Denoising autoencoders

- Contractive autoencoders

- Stacked denoising autoencoders

- Deep autoencoders


### 2. 2. What are the differences between greedy search and beam search?
贪心搜索会在第一个词生成后，会根据训练好的语言模型挑选出最有可能的第一个词最为第二个。接着根据第二个词选出最有可能出现的第三个词。这种方法可能训练的时候的表现很好，但是最后的呈现出来的效果其实可能一般。

比如, Jane is visiting African in September. 和 Jane is going to be visiting Afica in September。这两句话中，第一句话更简洁。然而如果用贪心算法的话，可能因为is 后面更有可能会出现going，最后算法给出的是第二句话。还有如果用贪心算法的话，可能会导致出现的句子不太丰富，当然这取决于应用场景。

Beam search在预测的时候，假设词表大小为3，内容为a,b,c。beam size是2，decoder解码的时候：
1. 生成第1个词的时候，选择概率最大的2个词，假设为a,c，那么当前的2个序列就是a和c。
2. 生成第2个词的时候，将当前序列a和c, 分别与词表中的所有词进行组合，得到新的6个序列aa ab ac ca cb cc，计算每个序列的得分，并选择得分最高的2个序列，作为新的当前序列，假如为 aa cb。
3. 不断重复这个过程，直到遇到结束符或者达到最大长度为止。最终输出得分最高的2个序列。

所以beam search可以获得前beam size个概率最大的结果。而不是greedy search的单一的最大概率的那个结果。


### 2. 3. What is the intuition of attention mechanism?

在Encoder-Decoder结构中，Encoder把所有的输入序列都编码成一个统一的语义特征c再来解码，因此，c中必须包含原始序列中的所有信息，它的长度就成了限制模型性能的瓶颈。如机器翻译问题，当要翻译的句子较长时，一个c可能存不下那么多信息，就会造成翻译精度的下降。Attention机制通过在每个时间输入不同的c来解决这个问题。

这时，每一个c会自动去选取与当前所要输出的y最合适的上下文信息。具体来说，我们用权重值aij来衡量Encoder中第j阶段的hj和解码时第i阶段的相关性。最终Decoder中第i阶段的输入的上下文信息ci就来自于所有hj对aij的加权和。 

例如：

h1*a11 + h2*a12 + h3*a13 + h4*a14 = c1

h1*a21 + h2*a22 + h3*a23 + h4*a24 = c2

h1*a31 + h2*a32 + h3*a33 + h4*a34 = c3

而这些权重aij是在训练模型的时候得出来的。


来源：https://cloud.tencent.com/developer/article/1164493

### 2. 4. What is the disadvantage of word embeding introduced in previous lectures ?
word embedding 的缺点是，训练好之后每个单词的表达就固定住了，以后使用的时候，不论新句子上下文单词是什么，这个单词的 Word Embedding 不会跟着上下文场景的变化而改变，如：“ 我喜欢吃苹果 ”，“ 很多人觉得苹果手机很好用 ” 。这两个句子中的苹果是不同的语义，表示不同的对象，没有办法表示出来。

### 2. 5. Briefly describe what is self-attention and what is multi-head attention?

### self-attention

传统的Attention是基于source端和target端的隐变量（hidden state）计算Attention的，得到的结果是源端的每个词与目标端每个词之间的依赖关系。但Self Attention不同，它分别在source端和target端进行，仅与source input或者target input自身相关的Self Attention，捕捉source端或target端自身的词与词之间的依赖关系；然后再把source端的得到的self Attention加入到target端得到的Attention中，捕捉source端和target端词与词之间的依赖关系。因此，self Attention Attention比传统的Attention mechanism效果要好，主要原因之一是，传统的Attention机制忽略了源端或目标端句子中词与词之间的依赖关系，相对比，self Attention可以不仅可以得到源端与目标端词与词之间的依赖关系，同时还可以有效获取源端或目标端自身词与词之间的依赖关系。

来源：https://zhuanlan.zhihu.com/p/79115586

### multi-attention

Multi-Head Attention相当于h个不同的self-attention的集成（ensemble），在这里我们以h=8 举例说明。Multi-Head Attention的输出分成3步：

1. 将数据 X 分别输入到8个self-attention中，得到8个加权后的特征矩阵Zi, i 是1,2,3...,8 。

2. 将8个Zi按列拼成一个大的特征矩阵；

3. 特征矩阵经过一层全连接后得到输出Z。

## 3. 中英文自动翻译模型的构建（使用encoder-decoder模型）

![](https://media.geeksforgeeks.org/wp-content/uploads/seq2seq.png)

### 3.1 [中英文翻译数据集下载](http://www.manythings.org/anki/)
找到Chinese (Mandarin) - English cmn-eng.zip (22075条中英文翻译)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from keras.layers import Input, LSTM, Dense
from keras.models import Model, load_model
from keras.utils import plot_model

import pandas as pd
import numpy as np

Using TensorFlow backend.


In [0]:
def create_model(n_input, n_output, n_units):
    # encoder
    encoder_input = Input(shape=(None, n_input))
    encoder = LSTM(n_units, return_state=True)
    _,encoder_h, encoder_c = encoder(encoder_input)
    encoder_state = [encoder_h, encoder_c]
    
    
    # Set up the decoder, using `encoder_states` as initial state.
    decoder_input = Input(shape=(None, n_output))
    decoder = LSTM(n_units, return_sequences=True, return_state=True)
    decoder_output, _, _ = decoder(decoder_input,
                                   initial_state=encoder_state)
    decoder_dense = Dense(n_output, activation='softmax')
    decoder_output = decoder_dense(decoder_output)
    
    # Define the model 
    model = Model([encoder_input, decoder_input], decoder_output)
    
    # inference setup
    # encoder
    encoder_infer = Model(encoder_input, encoder_state)
    
    # decoder
    decoder_state_input_h = Input(shape=(n_units,))
    decoder_state_input_c = Input(shape=(n_units,))    
    decoder_state_input = [decoder_state_input_h, decoder_state_input_c] 
    
    decoder_infer_output, decoder_infer_state_h, decoder_infer_state_c = decoder(decoder_input,
                                                                                 initial_state=decoder_state_input)
    decoder_infer_state = [decoder_infer_state_h, decoder_infer_state_c]
    decoder_infer_output = decoder_dense(decoder_infer_output)
    
    decoder_infer = Model([decoder_input] + decoder_state_input,
                          [decoder_infer_output] + decoder_infer_state)
    
    return model, encoder_infer, decoder_infer

In [0]:
N_UNITS = 256
BATCH_SIZE = 64
EPOCH = 50
NUM_SAMPLES = 10000

In [0]:
data_path = '/content/drive/My Drive/HCLG/lesson5/cmn-eng/cmn.txt'
df = pd.read_table(data_path,header=None).iloc[:NUM_SAMPLES,:,]
print(df.shape)
df.columns=['targets', 'inputs', 'others']
# print(df.head(5))
df['targets'] = df['targets'].apply(lambda x: '\t'+x+'\n')
# print(df.head(5))
input_texts = df.inputs.values.tolist()
print(type(input_texts))
print(input_texts[0:5])

target_texts = df.targets.values.tolist()
print(target_texts[0:5])

# print('df.inputs.unique()')
# print(len(df.inputs.unique())) #--> 9578

# print('df.inputs.unique().sum()')
# print(df.inputs.unique().sum()) # --> Hi.Run.Wait!Hello!I try.I won!Oh no!Cheers!Got it?He ran.Hop in.I qui

# print('set(df.inputs.unique().sum())')
# print(set(df.inputs.unique().sum())) # --> {'u', 'H', 't', 'M', '7', 'y', 'e', 'K', 'J', 'k ....

# print(len(set(df.inputs.unique().sum())))  #--> 72

input_characters = sorted(list(set(df.inputs.unique().sum())))
print(len(input_characters))


target_characters = sorted(list(set(df.targets.unique().sum())))

(10000, 3)
<class 'list'>
['嗨。', '你好。', '你用跑的。', '等等！', '等一下！']
['\tHi.\n', '\tHi.\n', '\tRun.\n', '\tWait!\n', '\tWait!\n']
2559


In [0]:
INUPT_LENGTH = max([len(i) for i in input_texts])
OUTPUT_LENGTH = max([len(i) for i in target_texts])
INPUT_FEATURE_LENGTH = len(input_characters)
# print(INPUT_FEATURE_LENGTH) #--> 72
OUTPUT_FEATURE_LENGTH = len(target_characters)
# print(OUTPUT_FEATURE_LENGTH)  #--> 2561

### 3.2  数据处理：encoder的输入，decoder的输入与输出
1，句子转换为one-hot编码     
2，LSTM需要的三维输入[n_samples, timestamp, one-hot feature]

In [0]:
INUPT_LENGTH = max([len(i) for i in input_texts])
OUTPUT_LENGTH = max([len(i) for i in target_texts])
INPUT_FEATURE_LENGTH = len(input_characters)
# print(INPUT_FEATURE_LENGTH) #--> 72
OUTPUT_FEATURE_LENGTH = len(target_characters)
# print(OUTPUT_FEATURE_LENGTH)  #--> 2561

In [0]:
encoder_input = np.zeros((NUM_SAMPLES, INUPT_LENGTH, INPUT_FEATURE_LENGTH))
decoder_input = np.zeros((NUM_SAMPLES, OUTPUT_LENGTH, OUTPUT_FEATURE_LENGTH))
decoder_output = np.zeros((NUM_SAMPLES, OUTPUT_LENGTH, OUTPUT_FEATURE_LENGTH))

In [0]:
input_dict = {char:index for index,char in enumerate(input_characters)}
print(input_dict)
input_dict_reverse = {index:char for index,char in enumerate(input_characters)}
target_dict = {char:index for index,char in enumerate(target_characters)}
target_dict_reverse = {index:char for index,char in enumerate(target_characters)}

{' ': 0, '!': 1, '"': 2, '(': 3, ')': 4, ',': 5, '.': 6, '/': 7, '0': 8, '1': 9, '2': 10, '3': 11, '4': 12, '5': 13, '6': 14, '7': 15, '8': 16, '9': 17, ':': 18, '?': 19, 'A': 20, 'B': 21, 'C': 22, 'D': 23, 'F': 24, 'I': 25, 'J': 26, 'M': 27, 'O': 28, 'P': 29, 'T': 30, 'W': 31, 'a': 32, 'b': 33, 'c': 34, 'e': 35, 'f': 36, 'g': 37, 'h': 38, 'i': 39, 'j': 40, 'k': 41, 'l': 42, 'm': 43, 'o': 44, 'r': 45, 's': 46, 't': 47, 'w': 48, 'y': 49, '\u200b': 50, '‘': 51, '“': 52, '”': 53, '。': 54, '一': 55, '丁': 56, '七': 57, '万': 58, '丈': 59, '三': 60, '上': 61, '下': 62, '不': 63, '与': 64, '丐': 65, '丑': 66, '专': 67, '且': 68, '世': 69, '业': 70, '东': 71, '丝': 72, '丟': 73, '丢': 74, '两': 75, '严': 76, '並': 77, '丧': 78, '个': 79, '中': 80, '临': 81, '丹': 82, '为': 83, '主': 84, '丽': 85, '举': 86, '久': 87, '么': 88, '义': 89, '之': 90, '乌': 91, '乎': 92, '乏': 93, '乐': 94, '乘': 95, '九': 96, '乞': 97, '也': 98, '习': 99, '乡': 100, '书': 101, '买': 102, '乱': 103, '乳': 104, '乾': 105, '亂': 106, '了': 107, '予': 108, '争': 109, '事':

In [0]:
for seq_index,seq in enumerate(input_texts):
    for char_index, char in enumerate(seq):
        encoder_input[seq_index, char_index, input_dict[char]] = 1

In [0]:
for seq_index,seq in enumerate(target_texts):
    for char_index,char in enumerate(seq):
        decoder_input[seq_index,char_index, target_dict[char]] = 1.0
        if char_index > 0:
            decoder_output[seq_index,char_index-1, target_dict[char]] = 1.0

In [0]:
''.join([input_dict_reverse[np.argmax(i)] for i in encoder_input[0] if max(i) !=0])

'嗨。'

In [0]:
''.join([target_dict_reverse[np.argmax(i)] for i in decoder_output[0] if max(i) !=0])

'Hi.\n'

In [0]:
model_train, encoder_infer, decoder_infer = create_model(INPUT_FEATURE_LENGTH,
                                                         OUTPUT_FEATURE_LENGTH,
                                                         N_UNITS)

In [0]:
# Compile & run training
model_train.compile(optimizer='rmsprop',
                    loss='categorical_crossentropy')

In [0]:
model_train.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 2559)   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 74)     0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 256), (None, 2883584     input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  338944      input_2[0][0]                    
                                                                 lstm_1[0][1]               

In [0]:
encoder_infer.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, 2559)        0         
_________________________________________________________________
lstm_1 (LSTM)                [(None, 256), (None, 256) 2883584   
Total params: 2,883,584
Trainable params: 2,883,584
Non-trainable params: 0
_________________________________________________________________


In [0]:
decoder_infer.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, 74)     0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 256)          0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 256)          0                                            
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  338944      input_2[0][0]                    
                                                                 input_3[0][0]              

In [0]:
validation_split = 0.2
model_train.fit([encoder_input,decoder_input],
                decoder_output,
                batch_size=BATCH_SIZE,
                epochs=EPOCH,
                validation_split=validation_split)

Train on 8000 samples, validate on 2000 samples
Epoch 1/50
8000/8000 [==============================] - 66s 8ms/step - loss: 2.0262 - val_loss: 2.5529
Epoch 2/50
8000/8000 [==============================] - 69s 9ms/step - loss: 1.6854 - val_loss: 2.1510
Epoch 3/50
8000/8000 [==============================] - 69s 9ms/step - loss: 1.4857 - val_loss: 2.0075
Epoch 4/50
8000/8000 [==============================] - 68s 9ms/step - loss: 1.3881 - val_loss: 1.9159
Epoch 5/50
8000/8000 [==============================] - 68s 9ms/step - loss: 1.3065 - val_loss: 1.8460
Epoch 6/50
8000/8000 [==============================] - 68s 9ms/step - loss: 1.2292 - val_loss: 1.7459
Epoch 7/50
8000/8000 [==============================] - 67s 8ms/step - loss: 1.1663 - val_loss: 1.6620
Epoch 8/50
8000/8000 [==============================] - 68s 9ms/step - loss: 1.1180 - val_loss: 1.6078
Epoch 9/50
8000/8000 [==============================] - 68s 8ms/step - loss: 1.0723 - val_loss: 1.6047
Epoch 10/50
8000/8000 [==

### 3.3 encoder-decoder模型的搭建
1，模型训练    
2，模型推理     
3，模型预测，展示结果   

In [0]:
def predict_chinese(source,encoder_inference, decoder_inference, n_steps, features):
    state = encoder_inference.predict(source)
    predict_seq = np.zeros((1,1,features))
    predict_seq[0,0,target_dict['\t']] = 1

    output = ''

    for i in range(n_steps): # n_steps为句子最大长度
        yhat,h,c = decoder_inference.predict([predict_seq]+state)
        char_index = np.argmax(yhat[0,-1,:])
        char = target_dict_reverse[char_index]
        output += char
        state = [h,c]
        predict_seq = np.zeros((1,1,features))
        predict_seq[0,0,char_index] = 1
        if char == '\n':
            break
    return output

In [0]:
for i in range(1000,1100):
    test = encoder_input[i:i+1,:,:] 
    out = predict_chinese(test,encoder_infer,decoder_infer,OUTPUT_LENGTH,OUTPUT_FEATURE_LENGTH)
    print(input_texts[i])
    print(out)

停止发牢骚吧。
Stom gremsing.

停止抵抗！
Stop rellivet you ligs.

夏天过去了。
Semes are mather.

你可以慢慢来。
You can come in.

慢慢来。
Take atalle.

那样是错的。
That's a brother faim.

那是一個恥辱。
That's a bit day aroum.

那符合逻辑。
That's time.

停止发牢骚吧。
Stom gremsing.

停止抵抗！
Stop rellivet you ligs.

夏天过去了。
Semes are mather.

你可以慢慢来。
You can come in.

慢慢来。
Take atalle.

那样是错的。
That's a brother faim.

那是一個恥辱。
That's a bit day aroum.

那符合逻辑。
That's time.

那是我的大衣。
That's a breathe's hood.

那是完美的。
That's a good ideat.

太可惜了！
That's too bad.

多遗憾啊！
That's too bad.

那太糟糕了。
It's a sick tomorrow.

鳥兒歌唱。
Birds sing.

旗子升起了。
The ste is too saccer.

電話正在響。
The pen is for for this.

那是我的大衣。
That's a breathe's hood.

那是完美的。
That's a good ideat.

太可惜了！
That's too bad.

多遗憾啊！
That's too bad.

那太糟糕了。
It's a sick tomorrow.

鳥兒歌唱。
Birds sing.

旗子升起了。
The ste is too saccer.

電話正在響。
The pen is for for this.

他們目光相接。
Their eys are her aroum.

這些是筆。
These are thene fool.

他們恨湯姆。
They hated my car.

他們有工作。
They are very good news.

他們讓我走。
They

![](https://stickershop.line-scdn.net/stickershop/v1/product/3624648/LINEStorePC/main.png;compress=true)

### 以上是从中文到英文的翻译过程，过程的结果不是很好，参数有待调整。